In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# Pre-processing

In [ ]:
# Read the first few lines to inspect the file structure
file_path = 'transactions_dataset.csv'
sample = pd.read_csv(file_path, nrows=10)

# Display the sample
print(sample.head())


In [ ]:
file_path = 'transactions_dataset.csv'

chunk_size = 100000  # Adjust this based on your available memory
chunks_list = []
for chunk in pd.read_csv(file_path, delimiter=';', chunksize=chunk_size, dtype=str):

    # Convert any numerical columns to the correct dtype
    for col in ['sales_net', 'quantity']:
        chunk[col] = pd.to_numeric(chunk[col], errors='coerce', downcast='float')

    # Convert any date columns to the correct dtype
    for col in ['date_order', 'date_invoice']:
        chunk[col] = pd.to_datetime(chunk[col], errors='coerce')

    # One-hot encode categorical variables
    chunk = pd.get_dummies(chunk, columns=['order_channel'])

    # Append the processed chunk to the list
    chunks_list.append(chunk)

# Concatenate all processed chunks
data_df_processed = pd.concat(chunks_list, ignore_index=True)

data_df_processed.head()


In [ ]:
# Define the output file path for the entire DataFrame
output_file_path = 'data_df_processed_full.csv'

# Save the entire DataFrame to a CSV file
data_df_processed.to_csv(output_file_path, index=False)


In [ ]:
# Group by client_id and sum up sales_net to calculate LTV
client_ltv = data_df_processed.groupby('client_id')['sales_net'].sum().rename('LTV').reset_index()

In [ ]:
# Merge LTV back to the original dataset
data_df_processed = pd.merge(data_df_processed, client_ltv, on='client_id', how='left')

In [ ]:
# Feature engineering: Calculate the time since the last purchase
latest_date = data_df_processed['date_order'].max()
data_df_processed['days_since_last_purchase'] = (latest_date - data_df_processed['date_order']).dt.days

In [ ]:
# Scaling numerical variables
scaler = StandardScaler()
data_df_processed[['sales_net', 'quantity']] = scaler.fit_transform(data_df_processed[['sales_net', 'quantity']])

In [ ]:
# Extract date parts
data_df_processed['order_year'] = data_df_processed['date_order'].dt.year
data_df_processed['order_month'] = data_df_processed['date_order'].dt.month
data_df_processed['order_weekday'] = data_df_processed['date_order'].dt.weekday

In [ ]:
output_file_path = 'processed_transactions_dataset.csv'
data_df_processed.to_csv(output_file_path, index=False)

In [ ]:
data_df_processed.head()

# Sales Analysis

In [ ]:
sns.set(style="whitegrid")

# 1. Distribution of Customer Lifetime Value (LTV)
plt.figure(figsize=(10, 6))
sns.histplot(data_df_processed_full['LTV'], bins=50, kde=True)
plt.title('Distribution of Customer Lifetime Value (LTV)')
plt.xlabel('LTV')
plt.ylabel('Frequency')
plt.show()

# 2. Sales Over Time (Year, Month)

data_df_processed_full['year_month'] = data_df_processed_full['order_year'].astype(str) + '-' + data_df_processed_full['order_month'].astype(str)
sales_over_time = data_df_processed_full.groupby('year_month')['sales_net'].sum().reset_index()

plt.figure(figsize=(14, 7))
sns.lineplot(x='year_month', y='sales_net', data=sales_over_time)
plt.title('Sales Over Time')
plt.xlabel('Year and Month')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.show()



In [ ]:
for channel in ['order_channel_at the store', 'order_channel_by phone', 'order_channel_during the visit of a sales rep', 
                'order_channel_online', 'order_channel_other']:
    # Create new columns for total sales and quantity for each order channel
    data_df_processed_full[channel + '_total_sales'] = data_df_processed_full[channel] * data_df_processed_full['sales_net']
    data_df_processed_full[channel + '_total_quantity'] = data_df_processed_full[channel] * data_df_processed_full['quantity']

order_channel_avgs = pd.DataFrame({
    'Channel': ['at the store', 'by phone', 'during the visit of a sales rep', 'online', 'other'],
    'Avg Sales': [
        data_df_processed_full['order_channel_at the store_total_sales'].sum() / data_df_processed_full['order_channel_at the store'].sum(),
        data_df_processed_full['order_channel_by phone_total_sales'].sum() / data_df_processed_full['order_channel_by phone'].sum(),
        data_df_processed_full['order_channel_during the visit of a sales rep_total_sales'].sum() / data_df_processed_full['order_channel_during the visit of a sales rep'].sum(),
        data_df_processed_full['order_channel_online_total_sales'].sum() / data_df_processed_full['order_channel_online'].sum(),
        data_df_processed_full['order_channel_other_total_sales'].sum() / data_df_processed_full['order_channel_other'].sum(),
    ],
    'Avg Quantity': [
        data_df_processed_full['order_channel_at the store_total_quantity'].sum() / data_df_processed_full['order_channel_at the store'].sum(),
        data_df_processed_full['order_channel_by phone_total_quantity'].sum() / data_df_processed_full['order_channel_by phone'].sum(),
        data_df_processed_full['order_channel_during the visit of a sales rep_total_quantity'].sum() / data_df_processed_full['order_channel_during the visit of a sales rep'].sum(),
        data_df_processed_full['order_channel_online_total_quantity'].sum() / data_df_processed_full['order_channel_online'].sum(),
        data_df_processed_full['order_channel_other_total_quantity'].sum() / data_df_processed_full['order_channel_other'].sum(),
    ]
})

# Plot the average sales by order channel
plt.figure(figsize=(14, 7))
sns.barplot(x='Channel', y='Avg Sales', data=order_channel_avgs)
plt.title('Average Sales by Order Channel')
plt.xlabel('Order Channel')
plt.ylabel('Average Sales Net')
plt.show()

# Plot the average quantity by order channel
plt.figure(figsize=(14, 7))
sns.barplot(x='Channel', y='Avg Quantity', data=order_channel_avgs)
plt.title('Average Quantity by Order Channel')
plt.xlabel('Order Channel')
plt.ylabel('Average Quantity')
plt.show()


# Refund Analysis

In [ ]:
returns_or_refunds = data_df_processed_full[data_df_processed_full['sales_net'] < 0]

negative_quantity = data_df_processed_full[data_df_processed_full['quantity'] < 0]

In [ ]:
# Looking for any large transactions that might be data entry errors
quantity_threshold = 10000  # Example threshold for quantity
sales_net_threshold = 10000  # Example threshold for sales_net
potential_data_entry_errors = data_df_processed_full[
    (data_df_processed_full['quantity'] > quantity_threshold) |
    (data_df_processed_full['sales_net'] > sales_net_threshold)
]

In [ ]:
# Looking for rows where 'sales_net' is negative but 'quantity' is positive, which could indicate discounts or adjustments
discounts_and_adjustments = data_df_processed_full[
    (data_df_processed_full['sales_net'] < 0) &
    (data_df_processed_full['quantity'] > 0)
]


In [ ]:
returns_or_refunds.to_csv('returns_or_refunds.csv', index=False)
negative_quantity.to_csv('negative_quantity.csv', index=False)
potential_data_entry_errors.to_csv('potential_data_entry_errors.csv', index=False)
discounts_and_adjustments.to_csv('discounts_and_adjustments.csv', index=False)

In [ ]:
print(f"Returns or Refunds: {returns_or_refunds.shape[0]}")
print(f"Negative Quantity: {negative_quantity.shape[0]}")
print(f"Potential Data Entry Errors: {potential_data_entry_errors.shape[0]}")
print(f"Discounts and Adjustments: {discounts_and_adjustments.shape[0]}")

In [ ]:
returns_or_refunds = pd.read_csv('returns_or_refunds.csv')
negative_quantity = pd.read_csv('negative_quantity.csv')
potential_data_entry_errors = pd.read_csv('potential_data_entry_errors.csv')
discounts_and_adjustments = pd.read_csv('discounts_and_adjustments.csv')

In [ ]:
sns.set_style('whitegrid')

# Histogram of sales_net for returns or refunds
plt.figure(figsize=(12, 6))
sns.histplot(returns_or_refunds['sales_net'], bins=30, kde=False)
plt.title('Distribution of Sales Net for Returns or Refunds')
plt.xlabel('Sales Net')
plt.ylabel('Frequency')
plt.show()

# Histogram of sales_net for discounts and adjustments
plt.figure(figsize=(12, 6))
sns.histplot(discounts_and_adjustments['sales_net'], bins=30, kde=False)
plt.title('Distribution of Sales Net for Discounts and Adjustments')
plt.xlabel('Sales Net')
plt.ylabel('Frequency')
plt.show()

# Time Series plot for returns or refunds
monthly_returns = returns_or_refunds.groupby('order_month')['sales_net'].sum()
plt.figure(figsize=(14, 7))
monthly_returns.plot(kind='bar')
plt.title('Monthly Total Sales Net for Returns or Refunds')
plt.xlabel('Month and Year')
plt.ylabel('Total Sales Net')
plt.xticks(rotation=90)
plt.show()

# Time Series plot for discounts and adjustments
monthly_adjustments = discounts_and_adjustments.groupby('order_month')['sales_net'].sum()
plt.figure(figsize=(14, 7))
monthly_adjustments.plot(kind='bar', color='orange')
plt.title('Monthly Total Sales Net for Discounts and Adjustments')
plt.xlabel('Month and Year')
plt.ylabel('Total Sales Net')
plt.xticks(rotation=90)
plt.show()

# Boxplot to look for outliers and the spread of returns or refunds
plt.figure(figsize=(12, 6))
sns.boxplot(x='order_month', y='sales_net', data=returns_or_refunds)
plt.title('Boxplot of Monthly Sales Net for Returns or Refunds')
plt.xlabel('Month and Year')
plt.ylabel('Sales Net')
plt.xticks(rotation=90)
plt.show()

# Boxplot for discounts and adjustments
plt.figure(figsize=(12, 6))
sns.boxplot(x='order_month', y='sales_net', data=discounts_and_adjustments, color='orange')
plt.title('Boxplot of Monthly Sales Net for Discounts and Adjustments')
plt.xlabel('Month and Year')
plt.ylabel('Sales Net')
plt.xticks(rotation=90)
plt.show()


Graph analysis: 
- For the distribution analysis most values are near zero, meaning that the refunds and discounts are in general small amounts compare to the total sales net, the problem is that the volume of is very high given the frequency.

Monthly Total Sales Net for Returns or Refunds:

- The bar chart indicates the total negative sales_net per month due to returns or refunds.
- The bars consistently show negative values each month, which could suggest a regular occurrence of returns or refunds.
- The consistency in the monthly totals implies that the returns or refunds are a regular part of the business cycle.

Monthly Total Sales Net for Discounts and Adjustments:

- This bar chart shows the total negative sales_net per month due to discounts and adjustments.
- Similar to returns and refunds, discounts and adjustments appear to be consistent month-to-month.

Boxplot of Monthly Sales Net for Returns or Refunds and Boxplot of Monthly Sales Net for Discounts and Adjustments:

- Both boxplots provide a visual summary of the distribution of sales_net values across different months.
- The central line in each box represents the median value, which is consistently near zero but negative.
- The 'whiskers' of the boxplot indicate the range of the data, and the points outside of the whiskers are potential outliers.
- The presence of outliers suggests that there are some months with particularly large returns or discounts.

# RFM Table Analysis

In [ ]:
data_df_processed_full['date_order'] = pd.to_datetime(data_df_processed_full['date_order'], errors='coerce')

latest_date = data_df_processed_full['date_order'].max() + pd.Timedelta(days=1)
rfm_table = data_df_processed_full.groupby('client_id').agg({
    'date_order': lambda x: (latest_date - x.max()).days,
    'client_id': 'count',
    'sales_net': 'sum'
}).rename(columns={'date_order': 'Recency', 'client_id': 'Frequency', 'sales_net': 'MonetaryValue'})

# RFM Score Calculation
rfm_table['R_Score'] = pd.qcut(rfm_table['Recency'], 4, ['1','2','3','4'])
rfm_table['F_Score'] = pd.qcut(rfm_table['Frequency'].rank(method='first'), 4, ['4','3','2','1'])
rfm_table['M_Score'] = pd.qcut(rfm_table['MonetaryValue'], 4, ['4','3','2','1'])

# Combine RFM scores
rfm_table['RFM_Segment'] = rfm_table['R_Score'].astype(str) + rfm_table['F_Score'].astype(str) + rfm_table['M_Score'].astype(str)
rfm_table['RFM_Score'] = rfm_table[['R_Score', 'F_Score', 'M_Score']].sum(axis=1)

rfm_table.head()


# Time Series Analysis

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

time_series_data = data_df_processed_full.groupby('date_order')['sales_net'].sum()
result = seasonal_decompose(time_series_data, model='additive', period=1)  # You might need to adjust the period based on your data's seasonality

# Plot the decomposed components of the time series
result.plot()
plt.show()


# Top Product Volume Analysis

In [ ]:
product_performance = data_df_processed_full.groupby('product_id').agg({
    'quantity': 'sum', 
    'sales_net': 'sum'  
}).sort_values(by='quantity', ascending=False)

product_performance['quantity'].head(10).plot(kind='bar')
plt.title('Top 10 Products by Quantity Sold')
plt.xlabel('Product ID')
plt.ylabel('Quantity Sold')
plt.show()


# Top Product Revenue Analysis

In [ ]:
profitability_analysis = data_df_processed_full.groupby('product_id').agg({
    'sales_net': 'sum'
}).sort_values(by='sales_net', ascending=False)

profitability_analysis['sales_net'].head(10).plot(kind='bar', color='green')
plt.title('Top 10 Profitable Products')
plt.xlabel('Product ID')
plt.ylabel('Sales')
plt.show()


# Customer Lifetime Value Prediction Model

In [ ]:
ltv_per_customer = data_df_processed_full.groupby('client_id')['LTV'].mean().reset_index()
rfm_table_with_ltv = pd.merge(rfm_table, ltv_per_customer, on='client_id', how='left')

X = rfm_table_with_ltv[['Recency', 'Frequency', 'MonetaryValue']]
y = rfm_table_with_ltv['LTV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
model = LinearRegression()

model.fit(X_train, y_train)
predicted_LTV = model.predict(X_test)


In [ ]:
r2 = r2_score(y_test, predicted_LTV)
mae = mean_absolute_error(y_test, predicted_LTV)
mse = mean_squared_error(y_test, predicted_LTV)
rmse = np.sqrt(mse)

print(f'R-squared: {r2}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error: {mse}')
print(f'Root Mean Squared Error: {rmse}')

In [ ]:
# Actual vs predicted values
plt.scatter(y_test, predicted_LTV)
plt.xlabel('Actual LTV')
plt.ylabel('Predicted LTV')
plt.title('Actual vs. Predicted LTV')
plt.show()

# Residuals
residuals = y_test - predicted_LTV
plt.scatter(y_test, residuals)
plt.xlabel('Actual LTV')
plt.ylabel('Residuals')
plt.title('Actual LTV vs. Residuals')
plt.axhline(y=0, color='red', linestyle='--')
plt.show()


# Outlier Detection

In [ ]:
data_df_processed_full['z_scores'] = stats.zscore(data_df_processed_full['sales_net'])
threshold = 3

data_df_processed_full['outlier'] = data_df_processed_full['z_scores'].apply(lambda x: x > threshold or x < -threshold)

# Plot of Sales Outliers
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data_df_processed_full, x=data_df_processed_full.index, y='sales_net', hue='outlier', style='outlier', palette=['blue', 'red'])
plt.title('Outliers in Sales Net')
plt.xlabel('Index')
plt.ylabel('Sales Net')
plt.legend(title='Outlier')
plt.show()


# Basic Churn Prediction Model

In [ ]:
rfm_table['churned'] = rfm_table['Recency'] > 90  # True if churned, False if not

X = rfm_table[['Recency', 'Frequency', 'MonetaryValue']]
y = rfm_table['churned'].astype(int)  # Converting boolean
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


model = RandomForestClassifier(random_state=0)
model.fit(X_train, y_train)
predicted_churn = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, predicted_churn))
print(classification_report(y_test, predicted_churn))


In [ ]:
order_channels = [col for col in data_df_processed_full.columns if 'order_channel_' in col]
melted_df = data_df_processed_full.melt(id_vars=['branch_id', 'client_id'], value_vars=order_channels, var_name='order_channel', value_name='order_flag')
melted_df = melted_df[melted_df['order_flag'] > 0]
orders_per_branch_channel = melted_df.groupby(['branch_id', 'order_channel'])['client_id'].count().reset_index(name='number_of_orders')
pivot_table = orders_per_branch_channel.pivot_table(index='branch_id', columns='order_channel', values='number_of_orders', fill_value=0)

pivot_table

In [ ]:
pivot_table.to_csv('orders_per_branch_channel.csv')

In [ ]:
# Branches with only one channel
branch_channel_sums = pivot_table.sum(axis=1)
single_channel_branches = branch_channel_sums[branch_channel_sums == 1]

single_channel_branches

In [ ]:
order_channels = [col for col in data_df_processed_full.columns if 'order_channel_' in col]

for channel in order_channels:
    data_df_processed_full[channel + '_sales'] = data_df_processed_full[channel] * data_df_processed_full['sales_net']

sales_columns = [col for col in data_df_processed_full.columns if '_sales' in col]
sales_per_branch_channel = data_df_processed_full.groupby('branch_id')[sales_columns].sum().reset_index()

sales_per_branch_channel

In [ ]:
sales_per_branch_channel.to_csv('sales_per_branch_channel.csv')

In [ ]:
# Highest income channel for each branch
sales_per_branch_channel['highest_income_channel'] = sales_per_branch_channel[sales_columns].idxmax(axis=1)
sales_per_branch_channel['highest_income'] = sales_per_branch_channel[sales_columns].max(axis=1)

sales_per_branch_channel[['branch_id', 'highest_income_channel', 'highest_income']]


In [ ]:
refunds = data_df_processed_full[data_df_processed_full['sales_net'] < 0]

In [ ]:
purchases = pd.read_csv('purchases.csv') 

In [ ]:
data_df_processed_full['date_order'] = pd.to_datetime(data_df_processed_full['date_order'])
latest_purchase_by_customer = data_df_processed_full.groupby('client_id')['date_order'].max().reset_index()
latest_purchase_by_customer.rename(columns={'date_order': 'latest_purchase_date'}, inplace=True)


In [ ]:
most_recent_date = data_df_processed_full['date_order'].max()
churn_threshold_date = most_recent_date - pd.Timedelta(days=90)

In [ ]:
latest_purchase_by_customer['churned'] = latest_purchase_by_customer['latest_purchase_date'].apply(lambda x: 1 if x < churn_threshold_date else 0)

In [ ]:
latest_purchase_by_customer.head()

# Churning Products Analysis

In [ ]:
latest_purchase_by_customer.to_csv('latest_purchase_by_customer.csv')

In [ ]:
purchases.head()

In [ ]:
# Identify product_ids associated with refunds.
refunded_products = refunds[['client_id', 'product_id']].drop_duplicates()

# Checking for those product_ids in the purchases data.
purchased_and_refunded = purchases.merge(refunded_products, on=['client_id', 'product_id'], how='inner')

# Marking these customers in your latest_purchase_by_customer DataFrame.
latest_purchase_by_customer['purchased_and_refunded'] = latest_purchase_by_customer['client_id'].isin(purchased_and_refunded['client_id']).astype(int)

# Checking for churn among these customers.
mask = latest_purchase_by_customer['purchased_and_refunded'] == 1

# Calculating the rate of churn
churn_rate = latest_purchase_by_customer.loc[mask, 'churned'].mean()

print(f"Churn rate among customers who purchased and refunded: {churn_rate:.2%}")


In [ ]:
purchased_and_refunded_with_churn = purchased_and_refunded.merge(
    latest_purchase_by_customer[['client_id', 'churned']],
    on='client_id',
    how='left'
)

churned_transactions = purchased_and_refunded_with_churn[purchased_and_refunded_with_churn['churned'] == 1]
churned_product_counts = churned_transactions['product_id'].value_counts().reset_index()
churned_product_counts.columns = ['product_id', 'churned_count']

# Products with highest churn
print(churned_product_counts)


In [ ]:
top_n = 10
top_churned_products = churned_product_counts.head(top_n)
plt.figure(figsize=(12, 8))  # You can adjust the size as needed
sns.barplot(
    data=top_churned_products,
    x='product_id',
    y='churned_count',
    palette='viridis'
)

plt.title('Top Products Leading to Customer Churn After a Refund')
plt.xlabel('Product ID')
plt.ylabel('Number of Churned Customers')
plt.xticks(rotation=45)  # Rotate the x labels to fit longer product ids if necessary

# Product that yield to customer churn
plt.show()


In [ ]:
top_churned_product_ids = churned_product_counts['product_id'].head(top_n).tolist()
top_products_sales_data = data_df_processed_full[
    data_df_processed_full['product_id'].isin(top_churned_product_ids)
].copy()

top_products_sales_data['year'] = top_products_sales_data['date_order'].dt.year
yearly_sales = top_products_sales_data.groupby(['year', 'product_id'])['sales_net'].sum().reset_index()

plt.figure(figsize=(14, 7))
sns.lineplot(data=yearly_sales, x='year', y='sales_net', hue='product_id', marker='o')

plt.title('Yearly Sales Trend of Top Products Leading to Customer Churn')
plt.xlabel('Year')
plt.ylabel('Total Sales Net')
plt.legend(title='Product ID')
plt.xticks(yearly_sales['year'].unique())  # Show all years in the dataset

# Churning products purchasing trends
plt.show()


# Branch sales and Channel Analysis

In [ ]:
sales_by_branch_channel = data_df_processed_full.groupby('branch_id').agg({
    'order_channel_at the store': 'sum',
    'order_channel_by phone': 'sum',
    'order_channel_online': 'sum',
    'order_channel_during the visit of a sales rep': 'sum',
    'order_channel_other': 'sum',
}).reset_index()

sales_by_branch_channel['total_sales'] = sales_by_branch_channel.sum(axis=1)
sales_by_branch_channel['online_phone_sales'] = sales_by_branch_channel[['order_channel_by phone', 'order_channel_online']].sum(axis=1)
sales_by_branch_channel['online_phone_proportion'] = sales_by_branch_channel['online_phone_sales'] / sales_by_branch_channel['total_sales']

threshold = 0.8
branches_high_reliance_online_phone = sales_by_branch_channel[sales_by_branch_channel['online_phone_proportion'] >= threshold]

print(branches_high_reliance_online_phone[['branch_id', 'online_phone_proportion']])


In [ ]:
branches_high_reliance_online_phone.to_csv('branches_high_reliance_online_phone.csv')

In [ ]:
sns.set_style("whitegrid")

plot_data = branches_high_reliance_online_phone.set_index('branch_id')[['order_channel_by phone', 'order_channel_online']]
plot_data['other_channels'] = 1 - plot_data.sum(axis=1)  # Calculate the proportion of other channels

plot_data.sort_values('other_channels').plot(
    kind='barh', 
    stacked=True, 
    figsize=(10, 7), 
    colormap='coolwarm'
)

plt.title('Proportion of Sales by Channel for Each Branch')
plt.xlabel('Proportion of Sales')
plt.ylabel('Branch ID')
plt.legend(title='Sales Channel', loc='center left', bbox_to_anchor=(1, 0.5))

# Branch sales by channel distribution
plt.tight_layout()
plt.show()
